In [8]:
import json
import os
import re
from tqdm import tqdm
from sentence_transformers import CrossEncoder

# 데이터 경로 (os.path.join 사용, 절대 경로)
BASE_DIR = os.path.dirname(os.getcwd())  # ml_code 폴더의 상위 폴더로 이동
DATA_DIR = os.path.join(BASE_DIR, 'data', 'ML')
os.makedirs(DATA_DIR, exist_ok=True)
print(BASE_DIR)
print(DATA_DIR)

c:\final_git\SKN12-FINAL-5TEAM
c:\final_git\SKN12-FINAL-5TEAM\data\ML


In [ ]:

# BASE_DIR = os.path.abspath(os.path.dirname(__file__))
TRAINING_DATA_PATH = os.path.join(BASE_DIR, 'data', '채용면접 인터뷰 데이터', 'Training')
VALIDATION_DATA_PATH = os.path.join(BASE_DIR,'data', '채용면접 인터뷰 데이터', 'Validation')
OUTPUT_FILE_PATH = os.path.join(BASE_DIR,'data', 'preprocessed_data.json')

print(f"Training data path: {TRAINING_DATA_PATH}")

Training data path: c:\final_git\SKN12-FINAL-5TEAM\data\채용면접 인터뷰 데이터\Training


In [3]:
# 전처리 파라미터
MIN_SENTENCE_LENGTH = 5  # 이 길이 미만의 문장은 필터링

# 제거할 추임새 및 습관어 목록 (정규식으로 구성)
# 주의: 단어 경계(\b)를 사용하여 부분 일치를 방지 (예: '그'가 '그림'에서 삭제되는 것을 방지)
FILLER_WORDS_PATTERN = re.compile(
    r'\b(음|네|아|어|그|저|뭐랄까|그러니까|사실|그냥|약간|일단|아무튼|솔직히|어떻게 보면|좀|너무|글쎄요|있잖아요|뭐라고 해야 할까|노력하는 편입니다|잘 모르겠지만)\b'
)

# 제거할 불필요한 기호 목록
SYMBOL_PATTERN = re.compile(r'[\"()[\\].,?!...]{2,}') # 2번 이상 반복되는 기호
WHITESPACE_PATTERN = re.compile(r'\s+') # 중복 공백
# model = CrossEncoder("BM-K/KoSimCSE-roberta", device='cuda', max_length=512)
model = CrossEncoder("BM-K/KoSimCSE-roberta", max_length=512)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BM-K/KoSimCSE-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def remove_fillers(text):
    """추임새 및 습관어를 제거합니다."""
    return FILLER_WORDS_PATTERN.sub('', text).strip()

def remove_symbols(text):
    """불필요한 기호를 제거합니다."""
    return SYMBOL_PATTERN.sub(' ', text)

def normalize_whitespace(text):
    """중복 공백 및 앞뒤 공백을 정리합니다."""
    return WHITESPACE_PATTERN.sub(' ', text).strip()

def preprocess_text(text):
    """정의된 파이프라인에 따라 텍스트를 전처리합니다."""
    if not isinstance(text, str):
        return ""
    
    # 1. 추임새 및 습관어 제거
    text = remove_fillers(text)
    # 2. 불필요한 기호 제거
    text = remove_symbols(text)
    # 3. 공백 정리
    text = normalize_whitespace(text)
    # 4. (선택) 맞춤법 교정
    # text = correct_spelling(text)
    
    return text

def compute_similarity(question, answer, model):
    # CrossEncoder는 쌍으로 넣음
    return float(model.predict([(question, answer)])[0])

def normalize_score(score, min_score, max_score):
    return (score - min_score) / (max_score - min_score)

def save_json(data, output_path):
    """데이터를 JSON 파일로 저장합니다."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# 감정 expression → 점수 매핑
def get_emotion_score(expression):
    positive = ["p-achievement", "p-affection", "p-gratitude", "p-happiness", "p-interest"]
    negative = ["n-anxiety", "n-distress", "n-sadness"]
    neutral = ["u-belief", "u-fact"]

    if expression in positive:
        return 0.25
    elif expression in negative:
        return -0.25
    elif expression in neutral or expression == "":
        return 0
    else:
        return 0

# 요약 비율 점수
def score_summary_ratio(summary_wc, answer_wc):
    if answer_wc == 0:
        return 0
    ratio = summary_wc / answer_wc
    if ratio < 0.1 or ratio > 0.5:
        return 0
    elif 0.2 <= ratio <= 0.35:
        return 1
    elif 0.1 <= ratio < 0.2:
        return round((1 * (ratio - 0.1) / 0.1), 2)
    elif 0.35 < ratio <= 0.5:
        return round((1 * (0.5 - ratio) / 0.15), 2)
    else:
        return 0


In [6]:
def get_all_file_paths(directory_paths):
    """모든 JSON 파일 경로를 재귀적으로 찾습니다."""
    file_paths = []
    for path in directory_paths:
        if not os.path.exists(path):
            print(f"경로가 존재하지 않습니다: {path}")
            continue
        for root, _, files in os.walk(path):
            for file in files:
                if file.endswith('.json'):
                    file_paths.append(os.path.join(root, file))
    return file_paths

def preprocess():
    """전체 전처리 과정을 실행하고 결과를 파일로 저장합니다."""
    print("면접 텍스트 데이터 전처리를 시작합니다.")
    
    # 1. 데이터 로딩
    all_json_files = get_all_file_paths([TRAINING_DATA_PATH, VALIDATION_DATA_PATH])
    if not all_json_files:
        print("오류: 지정된 경로에서 JSON 파일을 찾을 수 없습니다.")
        return

    print(f"총 {len(all_json_files)}개의 파일을 발견했습니다. 전처리를 진행합니다...")
    
    processed_data = []
    
    for file_path in tqdm(all_json_files, desc="전처리 진행 중"):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"JSON 디코딩 오류: {file_path}: {e}")
                    continue
                # 2. 원본 텍스트 추출
                raw_question = data.get('dataSet', {}).get('question', {}).get('raw', {}).get('text', "")
                raw_answer = data.get('dataSet', {}).get('answer', {}).get('raw', {}).get('text', "")
                raw_answer_count = data.get('dataSet', {}).get('answer', {}).get('raw', {}).get('wordCount', "")
                raw_answer_summary_count = data.get('dataSet', {}).get('answer', {}).get('summary', {}).get('wordCount', "")
                raw_expression = data.get('dataSet', {}).get('answer', {}).get('intent', [])[0].get('expression', "")
                # 3. 전처리 적용
                clean_question = preprocess_text(raw_question)
                clean_answer = preprocess_text(raw_answer)
                
                # 4. 유효성 검사 (Null, 빈 문자열, 최소 길이)
                if not clean_question or not clean_answer:
                    continue
                if len(clean_question) < MIN_SENTENCE_LENGTH or len(clean_answer) < MIN_SENTENCE_LENGTH:
                    continue
                # 5. 최종 데이터 구조화
                file_id = os.path.basename(file_path).split('.')[0]
                processed_data.append({
                    "id": file_id,
                    "question": clean_question,
                    "answer": clean_answer,
                    "answer_word_count": raw_answer_count,
                    "answer_summary_word_count": raw_answer_summary_count,
                    "expression": raw_expression,
                })
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")
        except Exception as e:
            print(f"알 수 없는 오류 발생 {file_path}: {e}")

    print(f"전처리 완료. 총 {len(processed_data)}개의 유효한 데이터가 처리되었습니다.")
    return processed_data

def add_similarity_scores_to_json(data, model):
    # data = json.load(data)
    print("유사도 점수 계산을 시작합니다...")
    updated_data = []
    for item in tqdm(data, desc="유사도 점수 계산 중", ncols=100):
        question = item["question"]
        answer = item["answer"]
        score = compute_similarity(question, answer, model)

        # 점수 추가
        item["similarity_score"] = round(score, 4)
        updated_data.append(item)


    max_score = max(item["similarity_score"] for item in updated_data)
    min_score = min(item["similarity_score"] for item in updated_data)

    for item in tqdm(updated_data, desc="유사도 점수 정규화 중", ncols=100):
        item["normalized_similarity_score"] = round(normalize_score(item["similarity_score"], min_score, max_score), 4)
    

    del item["similarity_score"]

    print("유사도 점수 계산 완료.")
    return updated_data

def scoring(data):
    print("점수 계산을 시작합니다...")
    for item in tqdm(data, desc="최종 점수 계산 중", ncols=100):
        item['score'] = 0
        item['score'] += get_emotion_score(item['expression'])
        item['score'] += score_summary_ratio(item['answer_summary_word_count'], item['answer_word_count'])
        item['score'] += item['normalized_similarity_score']
        item['score'] = round(item['score'] * 100 / 3, 4)

    print("점수 계산 완료.")
    return data



In [7]:
def main():
    temp1 = preprocess()
    temp2 = add_similarity_scores_to_json(temp1, model)
    temp3 = scoring(temp2)
    save_json(temp3, OUTPUT_FILE_PATH)

In [21]:
main()

면접 텍스트 데이터 전처리를 시작합니다.
총 6572개의 파일을 발견했습니다. 전처리를 진행합니다...


전처리 진행 중: 100%|██████████| 6572/6572 [00:03<00:00, 2091.73it/s]


전처리 완료. 총 6572개의 유효한 데이터가 처리되었습니다.
유사도 점수 계산을 시작합니다...


유사도 점수 정규화 중: 100%|███████████████████████████████| 6572/6572 [00:00<00:00, 1328048.08it/s]


유사도 점수 계산 완료.
점수 계산을 시작합니다...


최종 점수 계산 중: 100%|████████████████████████████████████| 6572/6572 [00:00<00:00, 306294.42it/s]



점수 계산 완료.
